In [ ]:
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import fmin

import holoviews as hv; hv.extension("bokeh", logo=False)
hv.opts.defaults(hv.opts.Image(cmap="hot", width=600, colorbar=True))

In [ ]:
# Underdetermined
n = 20
m = 100
A = np.random.rand(n,m)
b = np.random.rand(n)

def two_norm(x):
    return np.linalg.norm(x,ord=2)

constr = ({'type': 'eq', 'fun': lambda x:  A @ x - b})
x0     = np.random.rand(m)
res    = minimize(two_norm, x0, method='SLSQP',constraints=constr)
x2     = res.x

def one_norm(x):
    return np.linalg.norm(x,ord=1)

res = minimize(one_norm, x0, method='SLSQP',constraints=constr)
x1  = res.x

In [ ]:
h=\
hv.Spikes((range(m),x2), kdims='x2', vdims='y2').opts(width=400, padding=0.001, line_width=2)+\
hv.Spikes((range(m),x1), kdims='x1', vdims='y1').opts(width=400, padding=0.001, line_width=2)+\
hv.Histogram( np.histogram(x2, 20, density=True) ).opts(fill_color='slateblue', alpha=0.4, width=400 )+\
hv.Histogram( np.histogram(x1, 20, density=True) ).opts(fill_color='slateblue', alpha=0.4, width=400 )

h.cols(2)

In [ ]:
# Overdetermined
n = 500
m = 100
A = np.random.rand(n,m)
b = np.random.rand(n)

xdag = np.linalg.pinv(A)@b

lam  = np.array([0, 0.1, 0.5])

def reg_norm(x,A,b,lam):
    return np.linalg.norm(A@x-b,ord=2) + lam*np.linalg.norm(x,ord=1)

h_list=[]
for j in range(len(lam)):
    res = minimize(reg_norm,args=(A,b,lam[j]),x0=xdag)
    x   = res.x
    h_list.extend([ hv.Spikes((range(m),x), kdims='index', vdims=f'lam={lam[j]}').opts(width=400, height=200, padding=0.001, line_width=2),
                    hv.Histogram( np.histogram(x, 20, density=True), kdims='x', vdims=hv.Dimension(f"{j}", label="Frequency") ).opts(fill_color='slateblue', alpha=0.4, width=400, height=200 )])

hv.Layout(h_list).cols(2)

In [ ]:
## Matrix Overdetermined System
n    = 300
m    = 60
p    = 20
A    = np.random.rand(n,m)
b    = np.random.rand(n,p)
lam  = np.array([0,0.1])

xdag = np.linalg.pinv(A)@b
xdag = xdag.reshape(-1)

def reg_norm_2d(x,A,b,lam,m,p):
    # scipy minimize() does not accept multidimensional x
    # To get around this, we pass x as a vector and reshape it
    x = np.reshape(x,(m,p))
    return np.linalg.norm(A@x-b,ord=2) + lam*np.linalg.norm(x,ord=1)

In [ ]:
for j in range(len(lam)):
    res = minimize(reg_norm_2d,args=(A,b,lam[j],m,p),x0=xdag)
    x   = res.x
    x   = x.reshape(m,p)
    display( hv.Image(x.T, vdims='z' ).opts(title=f'lam={lam[j]}'))